In [16]:
#Imports needed from pytorch
import torch
from torch.utils.data import Dataset
from collections import OrderedDict
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import SGD,Adam

#Some built-in imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from os.path import join, isfile
from os import listdir
import json

# SKLearn and Skorch
from sklearn.datasets import make_classification
from skorch import NeuralNet, NeuralNetClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import classification_report
from skorch.callbacks import EarlyStopping

#Imports from the repository
from data_processing import get_weights_matrix, get_tokens
import data_processing as dp
from privacy_policies_dataset import PrivacyPoliciesDataset as PPD
from database import Database

from urllib.parse import unquote
from bs4 import BeautifulSoup

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
def get_app_ids(policy_id_list):
    """Function to extract number of App IDs.

    :param policy_id_list: List of policy IDs.
    """
    
    # Establish a database connection
    database_connection = Database().get_database_connection()

    # Try with the database connection as a resource.
    with database_connection:
        with database_connection.cursor() as cursor:
            format_strings = ','.join(['%s'] * len(policy_id_list))
            cursor.execute("SELECT COUNT(DISTINCT(app_to_policy.app_store_id)) AS app_count FROM app_to_policy WHERE policy_id IN (%s)" % format_strings,
                            tuple(policy_id_list))

            # Fetch all app urls.
            result_rows = cursor.fetchall()
    
    number_of_apps = result_rows[0]['app_count']

    print(number_of_apps)
    
    return number_of_apps

In [21]:
with open('small-policies.txt', 'r') as filehandle:
    small_policies_list = [current_place.rstrip() for current_place in filehandle.readlines()]

In [7]:
get_app_ids(small_policies_list)

71046


71046

In [22]:
len(small_policies_list)

49936

In [23]:
with open('empty-policies.txt', 'r') as filehandle:
    empty_policies_list = [current_place.rstrip() for current_place in filehandle.readlines()]

In [11]:
get_app_ids(empty_policies_list)

94776


94776

In [24]:
len(empty_policies_list)

66150

In [25]:
with open('small-empty-policies.txt', 'r') as filehandle:
    small_empty_policies_list = [current_place.rstrip() for current_place in filehandle.readlines()]

In [14]:
get_app_ids(small_empty_policies_list)

60952


60952

In [26]:
len(small_empty_policies_list)

41350

In [30]:
small_or_empty_policies_combined = []
small_or_empty_policies_combined.extend(small_policies_list)
small_or_empty_policies_combined.extend(empty_policies_list)
small_or_empty_policies_combined.extend(small_empty_policies_list)
small_or_empty_policies_combined = list(set(small_or_empty_policies_combined))

In [31]:
len(small_or_empty_policies_combined)

157436

In [32]:
get_app_ids(small_or_empty_policies_combined)

226774


226774

In [33]:
with open('no-data-collection-policies.txt', 'r') as filehandle:
    no_data_collection_list = [current_place.rstrip() for current_place in filehandle.readlines()]

In [35]:
len(no_data_collection_list)

144903

In [34]:
no_data_collection_cleaned_list = [x for x in no_data_collection_list if x not in small_or_empty_policies_combined] 

In [36]:
len(no_data_collection_cleaned_list)

123935

In [40]:
get_app_ids(no_data_collection_cleaned_list)

191784


191784